In [2]:
import torch
import torch.nn as nn

import random
import numpy as np
import matplotlib.pyplot as plt
from mlxtend.data import loadlocal_mnist

In [3]:
def show(image, title):
    index = 1 
    plt.figure(figsize=(10,5))

    for x in zip(image, title):
        image = x[0]
        title = x[1]
        plt.subplot(2, 5, index)
        plt.imshow(image, cmap=plt.cm.gray)
        plt.title(x[1], fontsize = 9)
        index += 1
    plt.show()

def load_MNIST(images_filepath, labels_filepath, verbose=False):
    # training_images_filepath = 'train-images.idx3-ubyte'
    # training_labels_filepath = 'train-labels.idx1-ubyte'
    # test_images_filepath = 't10k-images.idx3-ubyte'
    # test_labels_filepath = 't10k-labels.idx1-ubyte'

    X_images, y_labels = loadlocal_mnist( images_filepath, labels_filepath)

    if verbose:
        print('Dimension of images:', np.shape(X_images))
        print('Dimension of labels:',np.shape(y_labels))

    # reshape the arrays:
    X_images = X_images.reshape(-1, 28, 28)

    if verbose:
        print('Dimension of images:', np.shape(X_images))

    if verbose:
        image = []
        title = []

        for i in range(0, 5):
            r = random.randint(1, len(X_images))
            image.append(X_images[r])
            title.append('image:' + str(y_labels[r]))

        show(image, title)

    return X_images, y_labels

In [ ]:
class myCNN(nn.Module):
    def __init__(self):
        super().__init__(self)
        self.conv1 = nn.Conv2d(1, 8, 3)
        self.conv2 = nn.Conv2d(8, 16, 3)
        self.conv3 = nn.Conv2d(16, 32, 3)

In [ ]:
l1 = nn.Conv2d(1, 8, 3)